In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
print(char_arr)

['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '단', '어', '나', '무', '놀', '이', '소', '녀', '키', '스', '사', '랑']


In [3]:
num_dic = {n: i for i, n in enumerate(char_arr)}
print(num_dic)

{'o': 17, '랑': 40, 'd': 6, 'r': 20, 'b': 4, '단': 29, 'g': 9, 'z': 28, 'v': 24, 'P': 2, 'S': 0, 'e': 7, 'c': 5, '키': 37, '어': 30, '사': 39, 'm': 15, 'i': 11, 'j': 12, '이': 34, 'p': 18, 't': 22, '무': 32, 'h': 10, 'u': 23, '나': 31, 'l': 14, '놀': 33, 'w': 25, 'a': 3, 'y': 27, 'q': 19, '녀': 36, 'E': 1, 'k': 13, 's': 21, 'f': 8, '스': 38, '소': 35, 'n': 16, 'x': 26}


In [4]:
dic_len = len(num_dic)
print(dic_len)

41


In [5]:
seq_data = [['word', '단어어'], ['wood', '나무무'],
            ['game', '놀이이'], ['girl', '소녀녀'],
            ['kiss', '키스스'], ['love', '사랑랑']]

In [6]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:        
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])        
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch

### 옵션 설정

In [7]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

### 신경망 모델 구성

In [8]:
# [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])

### 인코더 셀

In [9]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

### 디코더 셀

In [10]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                            initial_state=enc_states, dtype=tf.float32)

In [11]:
model = tf.layers.dense(outputs, n_class, activation=None)

In [12]:
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
logits=model, labels=targets))

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 신경망 모델 학습

In [1]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

print(target_batch)

NameError: name 'tf' is not defined

In [ ]:
#length check
input_check = []
for batch in input_batch:
    input_check

In [15]:
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 3.831133
Epoch: 0002 cost = 2.866802
Epoch: 0003 cost = 1.843194
Epoch: 0004 cost = 1.156194
Epoch: 0005 cost = 0.738397
Epoch: 0006 cost = 0.489085
Epoch: 0007 cost = 0.423022
Epoch: 0008 cost = 0.329646
Epoch: 0009 cost = 0.263023
Epoch: 0010 cost = 0.165349
Epoch: 0011 cost = 0.216159
Epoch: 0012 cost = 0.241344
Epoch: 0013 cost = 0.132908
Epoch: 0014 cost = 0.102196
Epoch: 0015 cost = 0.084905
Epoch: 0016 cost = 0.086248
Epoch: 0017 cost = 0.142764
Epoch: 0018 cost = 0.061367
Epoch: 0019 cost = 0.116928
Epoch: 0020 cost = 0.030388
Epoch: 0021 cost = 0.097610
Epoch: 0022 cost = 0.067882
Epoch: 0023 cost = 0.024071
Epoch: 0024 cost = 0.069247
Epoch: 0025 cost = 0.036232
Epoch: 0026 cost = 0.031283
Epoch: 0027 cost = 0.036504
Epoch: 0028 cost = 0.006239
Epoch: 0029 cost = 0.005352
Epoch: 0030 cost = 0.009490
Epoch: 0031 cost = 0.012215
Epoch: 0032 cost = 0.036421
Epoch: 0033 cost = 0.005149
Epoch: 0034 cost = 0.005768
Epoch: 0035 cost = 0.002909
Epoch: 0036 cost = 0

### 번역 테스트

In [16]:
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

In [17]:

print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어어
wodr -> 단무어
love -> 사랑랑
loev -> 사랑랑
abcd -> 소스녀어
